The aim of this notebook is to transform the tables used for compression into Arduino code. The code also generates the functions used to exploit these tables. The `frequency_table.cpp` and `frequency_table.h` files can be placed directly in the Arduino project directory for use.

## 1. Importations

In [29]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

#import logging
#logging.getLogger("tensorflow").setLevel(logging.ERROR)

# Linear algebra and data processing
import numpy as np
import pandas as pd
import math
import random

# Get version python/keras/tensorflow/sklearn
from platform import python_version
import sklearn

# Loading bar
from tqdm import tqdm

# Joblib
import joblib

# Others
import itertools
from scipy import linalg
import matplotlib as mpl

# Folder manipulation
import os
import sys

# Garbage collector
import gc

# For images augmentations
# import cv2

# For regex
import re

# Visualizaton
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import time

# Create defaultdict
from collections import defaultdict

## 2. Set parameters

In [30]:
HUFFMAN_DIR = "MODELS/HUFFMAN/"
RESULTS_DIR = "RESULTS/"
MODELS_DIR = "MODELS/"
MAIN_DIR = "./DATA/"

PROTO = "SMTP"

# Set keras types
#tf.keras.backend.set_floatx('float64')

# Context
LOOK_BACK_CONTEXT = 2 # On rajoute +1 car on prend le dernier paquet comme paquet à compresser...
LOOK_AHEAD_CONTEXT = 1 #TIMESTEPS
CONTEXT_SIZE = 2 # Nombre timesteps sur les contexts
CONTEXT_OUTPUT_SIZE = 30 # Size of contexte in model layer
# De preference < 128 car c'est la taille de la
# couche GRU avant

# Packet
LOOK_BACK_PACKET = 16
LOOK_AHEAD_PACKET = 1
QUANTITY_PACKET = 15000 # Check if similar to training !
NUM_SIN = 8
NUM_FEATS = NUM_SIN + 1

# Filter equipment
NB_EQUIP_MAX = 10 # Set to None if no equipment
NB_EQUIP_TRAIN = 5 # Set to None if no equipment
MODE_EQUIP = "train" #"train" # or "test" or None for both
MODEL_MODE_EQUIP = "train"
NB_EQUIP_LIMIT = NB_EQUIP_MAX # Limit to check to 
# see if we select the right number
# of equipment

# Learning parameters
SHUFFLE = False # For test generator
ALPHABET_SIZE = 2
BATCH_SIZE = 1 # For test generator

# Size added to header_length IN BYTES
EXTRA_SIZE = 0
CUSTOM_SIZE = 100 # Take the lead if extra size is define
CHECKSUM = True # True = Keep Checksum

# For evaluation parallelisation
BATCH_PACKET = 2

# For Huffman model only
CUT_VALUE = 6
KEEP_ERROR = True
OPTIMAL = True
SELECTIVE = True # if True he most rank bit is used (only one)
SOFT_MODE = False # True par default permet de prendre la moyenne si le champs est meconnu !
DECIMALS = None #None # Si None pas de rounding !

# Generated dataset parameters
LEFT_PADDING = True # Padding dataset

# Name
# min(LOOK_BACK_CONTEXT, CONTEXT_OUTPUT_SIZE) if CONTEXT_OUTPUT_SIZE == 0
# else LOOK_BACK_CONTEXT == 1 or 2 or 3 or 4...etc
# easy to set LOOK_BACK_CONTEXT == 0
FULL_NAME = f"LOSSLESS_CONTEXT{LOOK_BACK_CONTEXT}_PACKET{LOOK_BACK_PACKET}_SIN{NUM_SIN}_{PROTO}"

if (CHECKSUM):
        
    if (KEEP_ERROR):

        if (CUSTOM_SIZE is not None):
            EXT_NAME = f"_WITH_CHECKSUM_CUSTOM_SIZE{CUSTOM_SIZE}_MODE_EQUIP{MODE_EQUIP}_KEEP_ERROR_{CUT_VALUE}"
        else:
            EXT_NAME = f"_WITH_CHECKSUM_EXTRA_SIZE{EXTRA_SIZE}_MODE_EQUIP{MODE_EQUIP}_KEEP_ERROR_{CUT_VALUE}"

    else:

        if (CUSTOM_SIZE is not None):
            EXT_NAME = f"_WITH_CHECKSUM_CUSTOM_SIZE{CUSTOM_SIZE}_MODE_EQUIP{MODE_EQUIP}_{CUT_VALUE}"
        else:
            EXT_NAME = f"_WITH_CHECKSUM_EXTRA_SIZE{EXTRA_SIZE}_MODE_EQUIP{MODE_EQUIP}_{CUT_VALUE}"

else:
    
    if (KEEP_ERROR):

        if (CUSTOM_SIZE is not None):
            EXT_NAME = f"_CUSTOM_SIZE{CUSTOM_SIZE}_MODE_EQUIP{MODE_EQUIP}_KEEP_ERROR_{CUT_VALUE}"
        else:
            EXT_NAME = f"_EXTRA_SIZE{EXTRA_SIZE}_MODE_EQUIP{MODE_EQUIP}_KEEP_ERROR_{CUT_VALUE}"

    else:

        if (CUSTOM_SIZE is not None):
            EXT_NAME = f"_CUSTOM_SIZE{CUSTOM_SIZE}_MODE_EQUIP{MODE_EQUIP}_{CUT_VALUE}"
        else:
            EXT_NAME = f"_EXTRA_SIZE{EXTRA_SIZE}_MODE_EQUIP{MODE_EQUIP}_{CUT_VALUE}"
                

# If the SELECTIVE mode is activated 
if (SELECTIVE):
    EXT_NAME = EXT_NAME + "_SELECTIVE"


# If the OPTIMAL mode is activated 
if (OPTIMAL):
    EXT_NAME = EXT_NAME + "_OPTIMAL"


# If the LEFT_PADDING mode is activated 
if (LEFT_PADDING):
    EXT_NAME = EXT_NAME + "_LEFT_PADDING"


# Set the Seed for numpy random choice
np.random.seed(42)

## 3. Load data

In [121]:
df_huffman_groupby = pd.read_csv(f"{HUFFMAN_DIR}df_HUFFMAN_{FULL_NAME}{EXT_NAME}.csv")
arr = np.load(f"{HUFFMAN_DIR}arr_index_pos_HUFFMAN_{FULL_NAME}{EXT_NAME}.npy")

In [122]:
print(f"{HUFFMAN_DIR}df_HUFFMAN_{FULL_NAME}{EXT_NAME}.csv")

MODELS/HUFFMAN/df_HUFFMAN_LOSSLESS_CONTEXT2_PACKET16_SIN8_SMTP_WITH_CHECKSUM_CUSTOM_SIZE100_MODE_EQUIPtrain_KEEP_ERROR_6_SELECTIVE_OPTIMAL_LEFT_PADDING.csv


In [123]:
# CHANGE DATAFRAME FORMAT 
def my_func(x, max_length=8):
    value = str(x)
    length = len(value)
    value_extend = "0"*(max_length-length)
    value = value_extend + value
    return value 


df_huffman_groupby['key'] = df_huffman_groupby['key'].map(
    lambda x : my_func(x, max_length=CUT_VALUE))
df_huffman_groupby = df_huffman_groupby.groupby(
    ['ctx', 'pos', 'key']).mean()
key_range = 2**CUT_VALUE

In [124]:
# Change max position to use
# MAX_POS_PERCENT = 0.1 # percentage of max value
MAX_POS = 347 #int(MAX_POS_PERCENT*arr.shape[1])

# Update array
arr = arr[:, :MAX_POS, :]

# Update dataframe
l0 = df_huffman_groupby.index.get_level_values(0)
l1 = df_huffman_groupby.index.get_level_values(1)
cond = (l1.isin(np.arange(0, MAX_POS)))
df_huffman_groupby = df_huffman_groupby[cond]

In [125]:
arr.shape

(3, 347, 6)

In [126]:
position_array_size = arr.shape[0]*MAX_POS*arr.shape[2]
freqs_array_size = arr.shape[0]*MAX_POS*key_range

In [127]:
key_range

64

In [128]:
freqs_array_size

66624

In [129]:
position_array_size

6246

In [139]:
df_huffman_groupby.loc[1, 25]

,proba
key,
000000,0.826607
000001,0.246520
001000,0.024029
001001,0.951837
100000,0.002392
101000,0.630193
101001,0.626311


## 4. Write to file

### 4.1 frequency_table.cpp

In [120]:
with open("frequency_table.cpp", "w") as text_file:
    text_file.write("#include \"frequency_table.h\"\n\n")
    
    text_file.write("FrequencyTable::FrequencyTable():\n")
    
    text_file.write("  positionArray{\n")
    
    
    # SET POSITION ARRAY

    
    for i in range(arr.shape[0]):
        text_file.write("    {\n")
        for j in range(arr.shape[1]):
            text_file.write("        {")
            for k in range(arr.shape[2]):
                # -1 replace with 0 define the absence of position
                # so, -1+1 = 0
                text_file.write(f"{arr[i, j, k]+1}")
                
                if (k != (arr.shape[2]-1)):
                    text_file.write(",")
                    
            text_file.write("},\n") 
        text_file.write("      },\n")
    text_file.write("  },\n")
    
    
    ctx_shape = df_huffman_groupby \
                .index.get_level_values(0) \
                .value_counts() \
                .shape[0]
    pos_shape = df_huffman_groupby \
                    .index.get_level_values(1) \
                    .value_counts() \
                    .shape[0]
    key_range = 2**len(df_huffman_groupby.loc[
        0, 0].index[0])
    counters_context = [0]*ctx_shape
    counters_all = np.zeros((ctx_shape, pos_shape))
    
    
    # SET FREQS ARRAY
    
    # FOR UNDERSTANDING
    freqs_array = []
    freqs_array_ctx = []
    
    text_file.write("  freqsArray{\n")
    
    for i in range(ctx_shape):
    #for i in range(1, 2):
        
        freqs_array_ctx_tmp = []

        for j in tqdm(range(pos_shape)):
        #for j in tqdm(range(302, 303)):

            text_file.write("         ")
            
            # Check if the pos j exist
            try:
                size = int(df_huffman_groupby.loc[
                            i, j].index.values.size)
            except:
                size = 0
            
            init_range = 0
            end_range = init_range

            for k in range(size):

                # Extract key value
                key = int(df_huffman_groupby.loc[
                    i, j].index.values[k], 2)

                proba = df_huffman_groupby.loc[
                        i, j].values[k][0]

                end_range = key
                
                # On compte le nombre de proba == 0.5
                proba_eq_quantity = end_range - init_range
                if (proba_eq_quantity > 1):
                    
                    # Number can't be above 255 because we are uint8_t
                    # From 101 to 255 we have 154 value 
                    nb_value = int(np.ceil(
                        proba_eq_quantity / 154))
                    
                    proba_eq_values = [
                        154 for q in range(nb_value-1)]
                    proba_eq_values += [
                        proba_eq_quantity-((nb_value-1)*154)]
                    
                    for proba_eq in proba_eq_values:
                        proba_tmp = 0.5
                        proba_text = int(np.around(
                            proba_tmp,decimals=2, out=None)*100)
                        proba_eq_quantity_text = proba_eq+100 
                        # 100 = la base, le max que l'on peut avoir en proba
                        text_file.write(f"{proba_eq_quantity_text},")
                        
                        freqs_array.append(
                            proba_eq_quantity_text)
                        freqs_array_ctx_tmp.append(
                            proba_eq_quantity_text)
                        
                    counters_context[i] += nb_value
                    counters_all[i, j] += nb_value
                    
                    if (counters_all[i, j] > key_range): # 256 in case of CUT_VALUE = 8
                        print("[DEBUG] ERROR : ", i, " // ", j)
                        
                # Permet un gain de 1 octets on écrit 50 au lieu de 101
                elif ((proba_eq_quantity > 0) and (proba_eq_quantity < 2)): # proba_eq_quantity == 1
                    #print("[DEBUG] ELIF ! ")
                    
                    proba_tmp = 0.5
                    proba_text = int(np.around(
                        proba_tmp, decimals=2, out=None)*100) 
                    # 100 = la base, le max que l'on peut avoir en proba
                    text_file.write(f"{proba_text},")
                    
                    freqs_array.append(proba_text)
                    freqs_array_ctx_tmp.append(
                            proba_text)
                    
                    counters_context[i] += 1
                    counters_all[i, j] += 1
                    
                proba_text = int(
                    np.around(proba,decimals=2, out=None)*100)
                text_file.write(f"{proba_text},")
                
                freqs_array.append(proba_text)
                freqs_array_ctx_tmp.append(
                            proba_text)
                
                counters_context[i] += 1
                counters_all[i, j] += 1
                
                # Update init_range
                init_range = end_range + 1

            text_file.write("\n")
            
        text_file.write("   \n")
        
        freqs_array_ctx.append(freqs_array_ctx_tmp)
        
    text_file.write("  },\n")
    
    
    # SET FREQS COUNT
    
    
    freqs_count_array = []
    
    # Pas de count car il y a 256 valeurs... donc on stock le MAX
    text_file.write("  freqsCountArray{\n")
    
    for i in range(ctx_shape):
        
        text_file.write("    {")
        freqs_count_array_tmp = []
        
        for j in tqdm(range(pos_shape)): #pos_shape)):
            
            size = int(counters_all[i, j]) 

            text_file.write(f"{size},")
            freqs_count_array_tmp.append(size)

        text_file.write("},\n")
        freqs_count_array.append(
            freqs_count_array_tmp)
            
    text_file.write("  },\n")
    
    
    # SET FREQS COUNT CONTEXT ARRAY
    
    freqs_count_context_array = []

    text_file.write("  freqsCountContextArray{\n")
    text_file.write("    ")
    
    # Set first step
    counters_context = [0] + counters_context
    
    for i in range(1, ctx_shape+1):
        text_file.write(f"{sum(counters_context[0:i])},")
        freqs_count_context_array.append(
            sum(counters_context[0:i]))
        
    text_file.write("\n  }\n")
    
    text_file.write("{\n")
    
    text_file.write("  // Track the quantity\n")
    text_file.write(f"  sumCountFreqsArray = 0;\n")

    text_file.write("};\n\n")
    
    
    # getIndexFreqsArray
    
    
    text_file.write("int FrequencyTable::getIndexFreqsArray( \n \
      int contextSize, int position, int key)\n")
    text_file.write("{\n")
    
    text_file.write("  // Compute sum until position-1\n")
    text_file.write("  for (int i=0; i<position; i++)\n")
    text_file.write("  {\n")
    text_file.write("    sumCountFreqsArray = sumCountFreqsArray + \\\n \
                         freqsCountArray[contextSize][i];\n")
    text_file.write("  }\n\n")

    text_file.write("  uint8_t countFreqsArray = freqsCountArray[ \n \
                       contextSize][position];\n")
    
    text_file.write("  int indexFreqsArray = this->getIndexFreqsArray( \n \
                       countFreqsArray, key);\n\n")

    text_file.write("  return indexFreqsArray;\n")
    text_file.write("}\n\n")
    
    
    # getIndexFreqsArray (SURCHARGE)
    
    
    text_file.write("int FrequencyTable::getIndexFreqsArray( \n \
      int countFreqsArray, int key)\n")
    text_file.write("{\n")
    
    text_file.write("  int indexFreqsArray = -1;\n")
    text_file.write("  int count = 0;\n")
    text_file.write("  int countTmp = 0;\n")
    text_file.write("  bool exitLoop = false;\n\n")
    
    text_file.write("  int i = 0;\n")
    text_file.write("  while((i < countFreqsArray) && (!exitLoop))\n")
    text_file.write("  {\n")
    
    text_file.write("    // Count number of value\n")
    text_file.write("    countTmp = (int)freqsArray[sumCountFreqsArray+i];\n")
    text_file.write("    // Compteur for equal freq value\n")
    text_file.write("    count = count + max(1, countTmp-100);\n\n")
    
    text_file.write("    // -1 because index start from 0\n")
    text_file.write("    if ((count-1) >= key)\n")
    text_file.write("    {\n")
    text_file.write("      indexFreqsArray = sumCountFreqsArray+i;\n")
    text_file.write("      exitLoop = true;\n")
    text_file.write("    }\n\n")
    
    text_file.write("    i = i + 1;\n")
    text_file.write("  }\n")

    text_file.write("  sumCountFreqsArray = sumCountFreqsArray+countFreqsArray;\n")
    text_file.write("  return indexFreqsArray;\n")
    text_file.write("}\n\n")
    
    
    # convertBinToInt
    
    
    text_file.write("int FrequencyTable::convertBinToInt( \n \
                          uint8_t* dataBin, int sizeArrayBin)\n")
    text_file.write("{\n")
    text_file.write("int value = 0;\n\n")
    
    text_file.write("for(int i=0; i<sizeArrayBin; i++)\n")
    text_file.write("  {\n")
    
    text_file.write("    if (dataBin[i] == 1) // /!\ A CHANGER si bool !?\n")
    text_file.write("    {\n")
    text_file.write("    value = value + pow(\n")
    text_file.write("      2, sizeArrayBin-1-i);\n")
    text_file.write("    }\n")
    
    text_file.write("  }\n")
    text_file.write("  return value;\n")
    
    text_file.write("}\n\n")
    
    # computeKey
    
    text_file.write("int FrequencyTable::computeKey(uint8_t* dataBin, \n \
                          int context,\n \
                          int position)\n")
    text_file.write("{\n")
    
    text_file.write("  uint16_t* indexes = positionArray[\n")
    text_file.write("    context][position];\n")
    text_file.write("  uint8_t* keyArray = new uint8_t[KEY_SIZE];\n\n")
    
    text_file.write("  // Extract value of array \n")
    text_file.write("  for(int i=0; i<KEY_SIZE; i++)\n")
    text_file.write("  {\n")
    text_file.write("    // If index equal is -1 then indexes[i] == 0\n")
    text_file.write("    // because indexes[i] == -1 take too much space\n")
    text_file.write("    if(indexes[i] == 0)\n")
    text_file.write("    {\n")
    text_file.write("      keyArray[i] = 0;\n")
    text_file.write("    } else {\n")
    text_file.write("      keyArray[i] = dataBin[indexes[i]-1];\n")
    text_file.write("    }\n")
    text_file.write("  }\n\n")
    
    text_file.write("  // Convert binary array to int\n")
    text_file.write("  int key = this->convertBinToInt(\n")
    text_file.write("    keyArray, 8);\n")
    text_file.write("  delete[] keyArray;\n\n")
    
    text_file.write("  return key;\n")    
    
    text_file.write("}\n\n")
    
    
    # computeFrequency
    
    
    text_file.write("float* FrequencyTable::computeFrequency( \n \
          int indexFreqsArray)\n")
    text_file.write("{\n")
    
    text_file.write("  // Check if position is a compress field (eq > 100)\n")
    text_file.write("  // If number is on the trailling value\n")
    text_file.write("  float freq;\n")
    text_file.write("  if (indexFreqsArray == -1)\n")
    text_file.write("  {\n")
    text_file.write("    freq = 0.5;\n")
    text_file.write("  }\n")
    text_file.write("  else\n")
    text_file.write("  {\n")
    text_file.write("    freq = freqsArray[indexFreqsArray] / 100.;\n")
    text_file.write("  }\n\n")

    text_file.write("  // If number is on 100 \n")
    text_file.write("  float* freqs = new float[ALPHABET_SIZE];\n")
    text_file.write("  if (freq > 1)\n")
    text_file.write("  {\n")
    text_file.write("    freqs[0] = 0.5;\n")
    text_file.write("  }\n")

    text_file.write("  else\n")
    text_file.write("  {\n")
    text_file.write("    freqs[0] = freq;\n")
    text_file.write("  }\n\n")
  
    text_file.write("  freqs[1] = 1-freqs[0];\n")
    text_file.write("  return freqs;\n") 
    text_file.write("}\n\n")
    
    
    # nextFrequency
    
    
    text_file.write("float* FrequencyTable::nextFrequency(uint8_t* dataBin, \n \
                           int contextSize, \n \
                           int position) \n")
    text_file.write("{\n")
    
    text_file.write("  // Extract value from position\n")
    text_file.write("  int key = this->computeKey( \n \
        dataBin, contextSize, position);\n\n")

    text_file.write("  uint8_t countFreqsArray = freqsCountArray[ \n \
            contextSize][position];\n")
    text_file.write("  int indexFreqsArray = this->getIndexFreqsArray( \n \
          countFreqsArray, key);\n\n")

    text_file.write("  float* freqs = this->computeFrequency( \n \
                      indexFreqsArray);\n")
    text_file.write("  return freqs;\n")
    
    text_file.write("}\n\n")
    
    
    # getFrequency
    
    text_file.write("float* FrequencyTable::getFrequency(uint8_t* dataBin,\n \
                          int contextSize,\n \
                          int position)\n")
    text_file.write("{\n")
    
    text_file.write("  // Extract value from position\n")
    text_file.write(f"  int key = this->computeKey(\n")
    text_file.write(f"    dataBin, contextSize, position);\n\n")
    
    text_file.write(f"  // Init with pre-calculate context\n")
    text_file.write(f"  sumCountFreqsArray = freqsCountContextArray[ \n \
                           contextSize];\n")
    text_file.write(f"  int indexFreqsArray = this->getIndexFreqsArray( \n \
                           contextSize, position, key);\n\n")
    
    text_file.write(f"  float* freqs = this->computeFrequency( \n \
                       indexFreqsArray);\n")
    
    text_file.write(f"  return freqs;\n")
    
    text_file.write("}\n\n")

100%|██████████| 10/10 [00:00<00:00, 129453.83it/s]


In [48]:
len(freqs_array_ctx_tmp)

1573

In [49]:
# Each line for each context
for i in range(len(freqs_array_ctx)):
    print(len(freqs_array_ctx[i]))

1484
1500
1573


In [50]:
len(freqs_array)

4557

In [51]:
print(f"df_huffman_groupby.shape: {df_huffman_groupby.shape}")
print(f"uint8_t freqsArray[{int(counters_all.sum())}];")
print(f"uint8_t freqsCountArray[{arr.shape[0]}][{arr.shape[1]}];")
print(f"uint32_t freqsCountContextArray[{arr.shape[0]}];")

df_huffman_groupby.shape: (3703, 1)
uint8_t freqsArray[4557];
uint8_t freqsCountArray[3][100];
uint32_t freqsCountContextArray[3];


In [148]:
df_huffman_groupby.shape

(3703, 1)

In [149]:
print(f"uint8_t freqsArray[{int(counters_all.sum())}];")
print(f"uint8_t freqsCountArray[{arr.shape[0]}][{arr.shape[1]}];")
print(f"uint32_t freqsCountContextArray[{arr.shape[0]}];")

uint8_t freqsArray[4557];
uint8_t freqsCountArray[3][100];
uint32_t freqsCountContextArray[3];


### 4.2 frequency_table.h

In [150]:
with open("frequency_table.h", "w") as text_file:
    text_file.write("#ifndef FREQ_TABLE_H\n")
    text_file.write("#define FREQ_TABLE_H\n")
    text_file.write("#include <Arduino.h>\n\n")
    
    text_file.write("#define ALPHABET_SIZE     0x02\n")
    text_file.write(f"#define KEY_SIZE          0x0{CUT_VALUE}\n\n")
    
    text_file.write("class FrequencyTable\n")
    text_file.write("{\n")
    
    text_file.write("  private:\n")
    
    text_file.write(f"    uint16_t positionArray[{arr.shape[0]}][{arr.shape[1]}][{arr.shape[2]}];\n\n")
    
    # +10 is for safety
    text_file.write(f"    uint8_t freqsArray[{int(counters_all.sum()+key_range)}];\n")
    text_file.write(f"    uint8_t freqsCountArray[{arr.shape[0]}][{arr.shape[1]}];\n")
    text_file.write(f"    uint32_t freqsCountContextArray[{arr.shape[0]}];\n\n")
    
    text_file.write(f"    int indexFreqsArray;\n")
    text_file.write(f"    int sumCountFreqsArray;\n\n")

    
    text_file.write("    // Methods \n")
    text_file.write("    int convertBinToInt(uint8_t* dataBin, \n \
                           int sizeArrayBin);\n")
    text_file.write("    int computeKey(uint8_t* dataBin,\n \
                           int context,\n \
                           int position);\n")
    text_file.write("    int getIndexFreqsArray(int contextSize,\n \
                           int position,\n \
                           int key);\n")
    text_file.write("    int getIndexFreqsArray(int countFreqsArray,\n \
                           int key);\n")
    text_file.write(f"    float* computeFrequency(int nextPosition);\n\n")
    
    
    text_file.write("  public:\n")
    text_file.write("    // Constructor\n")
    text_file.write("    FrequencyTable();\n\n")
    
    text_file.write("    // Others\n")
    text_file.write("    float* nextFrequency(uint8_t* dataBin, \n \
                          int contextSize,\n \
                          int position);\n")
    text_file.write("    float* getFrequency(uint8_t* dataBin, \n \
                          int contextSize,\n \
                          int position);\n")
    
    text_file.write("    // Others\n")
    text_file.write("    // \n")
    
    text_file.write("};\n\n")
    
    text_file.write("#endif\n")